# UNC HiSeq mRNAseq gene expression (RSEM)

The goal of this notebook is to introduce you to the mRNAseq gene expression BigQuery table.

This table contains all available TCGA Level-3 gene expression data produced by UNC's RNAseqV2 pipeline using the Illumina HiSeq platform, as of July 2016.  The most recent archive (*eg* ``unc.edu_BRCA.IlluminaHiSeq_RNASeqV2.Level_3.1.11.0``) for each of the 33 tumor types was downloaded from the DCC, and data extracted from all files matching the pattern ``%.rsem.genes.normalized_results``. Each of these raw “RSEM genes normalized results” files has two columns: gene_id and normalized_count.  The gene_id string contains two parts: the gene symbol, and the Entrez gene ID, separated by **|**  *eg*: **`TP53|7157`**.  During ETL, the gene_id string is split and the gene symbol is stored in the ``original_gene_symbol`` field, and the Entrez gene ID is stored in the ``gene_id`` field.  In addition, the Entrez ID is used to look up the current HGNC approved gene symbol, which is stored in the ``HGNC_gene_sybmol`` field. 

In order to work with BigQuery, you need to import the python bigquery module (`gcp.bigquery`) and you need to know the name(s) of the table(s) you are going to be working with:

In [1]:
import gcp.bigquery as bq
mRNAseq_BQtable = bq.Table('isb-cgc:tcga_201607_beta.mRNA_UNC_HiSeq_RSEM')

From now on, we will refer to this table using this variable ($mRNAseq_BQtable), but we could just as well explicitly give the table name each time.

Let's start by taking a look at the table schema:

In [2]:
%bigquery schema --table $mRNAseq_BQtable

Now let's count up the number of unique patients, samples and aliquots mentioned in this table.  We will do this by defining a very simple parameterized query.  (Note that when using a variable for the table name in the FROM clause, you should not also use the square brackets that you usually would if you were specifying the table name as a string.)

In [3]:
%%sql --module count_unique

DEFINE QUERY q1
SELECT COUNT (DISTINCT $f, 25000) AS n
FROM $t

In [4]:
fieldList = ['ParticipantBarcode', 'SampleBarcode', 'AliquotBarcode']
for aField in fieldList:
  field = mRNAseq_BQtable.schema[aField]
  rdf = bq.Query(count_unique.q1,t=mRNAseq_BQtable,f=field).results().to_dataframe()
  print " There are %6d unique values in the field %s. " % ( rdf.iloc[0]['n'], aField)

 There are   9530 unique values in the field ParticipantBarcode. 
 There are  10289 unique values in the field SampleBarcode. 
 There are  10291 unique values in the field AliquotBarcode. 


We can do the same thing to look at how many unique gene symbols and gene ids exist in the table:

In [5]:
fieldList = ['original_gene_symbol', 'HGNC_gene_symbol', 'gene_id']
for aField in fieldList:
  field = mRNAseq_BQtable.schema[aField]
  rdf = bq.Query(count_unique.q1,t=mRNAseq_BQtable,f=field).results().to_dataframe()
  print " There are %6d unique values in the field %s. " % ( rdf.iloc[0]['n'], aField)

 There are  20501 unique values in the field original_gene_symbol. 
 There are  20182 unique values in the field HGNC_gene_symbol. 
 There are  20531 unique values in the field gene_id. 


Based on the counts, we can see that there are a few instances where the original gene symbol (from the underlying TCGA data file), or the HGNC gene symbol or the gene id (also from the original TCGA data file) is missing, but for the majority of genes, all three values should be available and for the most part the original gene symbol and the HGNC gene symbol that was added during ETL should all match up.  This next query will generate the complete list of genes for which none of the identifiers are null, and where the original gene symbol and the HGNC gene symbol match.  This list has over 18000 genes in it.

In [6]:
%%sql

SELECT
  HGNC_gene_symbol,
  original_gene_symbol,
  gene_id
FROM
  $mRNAseq_BQtable
WHERE
  ( original_gene_symbol IS NOT NULL
    AND HGNC_gene_symbol IS NOT NULL
    AND original_gene_symbol=HGNC_gene_symbol
    AND gene_id IS NOT NULL )
GROUP BY
  original_gene_symbol,
  HGNC_gene_symbol,
  gene_id
ORDER BY
  HGNC_gene_symbol

HGNC_gene_symbol,original_gene_symbol,gene_id
A1BG,A1BG,1
A1CF,A1CF,29974
A2M,A2M,2
A2ML1,A2ML1,144568
A4GALT,A4GALT,53947
A4GNT,A4GNT,51146
AAAS,AAAS,8086
AACS,AACS,65985
AADAC,AADAC,13
AADACL2,AADACL2,344752


We might also want to know how often the gene symbols do not agree:

In [7]:
%%sql

SELECT
  HGNC_gene_symbol,
  original_gene_symbol,
  gene_id
FROM
  $mRNAseq_BQtable
WHERE
  ( original_gene_symbol IS NOT NULL
    AND HGNC_gene_symbol IS NOT NULL
    AND original_gene_symbol!=HGNC_gene_symbol
    AND gene_id IS NOT NULL )
GROUP BY
  original_gene_symbol,
  HGNC_gene_symbol,
  gene_id
ORDER BY
  HGNC_gene_symbol

HGNC_gene_symbol,original_gene_symbol,gene_id
A1BG-AS1,NCRNA00181,503538
A2M-AS1,LOC144571,144571
AACSP1,AACSL,729522
AADACP1,LOC201651,201651
AAED1,C9orf21,195827
AAMDC,C11orf67,28971
AAR2,C20orf4,25980
AARD,C8orf85,441376
AATBC,LOC284837,284837
AATK-AS1,LOC388428,388428


BigQuery is not just a "look-up" service -- you can also use it to perform calculations.  In this next query, we take a look at the mean, standard deviation, and coefficient of variation for the expression of EGFR, within each tumor-type, as well as the number of primary tumor samples that went into each summary statistic.

In [8]:
%%sql

SELECT
  Study,
  n,
  exp_mean,
  exp_sigma,
  (exp_sigma/exp_mean) AS exp_cv
FROM (
  SELECT
    Study,
    AVG(LOG2(normalized_count+1)) AS exp_mean,
    STDDEV_POP(LOG2(normalized_count+1)) AS exp_sigma,
    COUNT(AliquotBarcode) AS n
  FROM
    $mRNAseq_BQtable
  WHERE
    ( SampleTypeLetterCode="TP"
      AND HGNC_gene_symbol="EGFR" )
  GROUP BY
    Study )
ORDER BY
  exp_sigma DESC

Study,n,exp_mean,exp_sigma,exp_cv
GBM,156,11.8977935121,2.37898710407,0.199951957617
SKCM,104,5.41189297221,2.3134143602,0.427468608873
BRCA,1095,7.3316523775,2.0186060124,0.275327567165
PCPG,179,5.21363387261,1.93511911652,0.371165134301
BLCA,408,9.26258869139,1.84003491022,0.19865233916
CESC,304,9.615363578,1.74670970515,0.181658206783
THYM,120,8.61408054406,1.74340125417,0.202389708948
LGG,516,11.1471994947,1.72222308803,0.154498274553
ESCA,184,11.6700063461,1.63953898439,0.140491696042
SARC,259,8.58641554296,1.59104767886,0.185298238933


We can also easily move the gene-symbol out of the WHERE clause and into the SELECT and GROUP BY clauses and have BigQuery do this same calculation over *all* genes and all tumor types.  This time we will use the `--module` option to define the query and then call it in the next cell from python.

In [9]:
%%sql --module highVar

SELECT
  Study,
  HGNC_gene_symbol,
  n,
  exp_mean,
  exp_sigma,
  (exp_sigma/exp_mean) AS exp_cv
FROM (
  SELECT
    Study,
    HGNC_gene_symbol,
    AVG(LOG2(normalized_count+1)) AS exp_mean,
    STDDEV_POP(LOG2(normalized_count+1)) AS exp_sigma,
    COUNT(AliquotBarcode) AS n
  FROM
    $t
  WHERE
    ( SampleTypeLetterCode="TP" )
  GROUP BY
    Study,
    HGNC_gene_symbol )
ORDER BY
  exp_sigma DESC

Once we have defined a query, we can put it into a python object and print out the SQL statement to make sure it looks as expected:

In [10]:
q = bq.Query(highVar,t=mRNAseq_BQtable)
print q.sql

SELECT
  Study,
  HGNC_gene_symbol,
  n,
  exp_mean,
  exp_sigma,
  (exp_sigma/exp_mean) AS exp_cv
FROM (
  SELECT
    Study,
    HGNC_gene_symbol,
    AVG(LOG2(normalized_count+1)) AS exp_mean,
    STDDEV_POP(LOG2(normalized_count+1)) AS exp_sigma,
    COUNT(AliquotBarcode) AS n
  FROM
    [isb-cgc:tcga_201607_beta.mRNA_UNC_HiSeq_RSEM]
  WHERE
    ( SampleTypeLetterCode="TP" )
  GROUP BY
    Study,
    HGNC_gene_symbol )
ORDER BY
  exp_sigma DESC


And then we can run it and save the results in another python object:

In [11]:
r = bq.Query(highVar,t=mRNAseq_BQtable).results()

In [12]:
#r.to_dataframe()

Since the result of the previous query is quite large (over 600,000 rows representing ~20,000 genes x ~30 tumor types), we might want to put those results into one or more subsequent queries that further refine these results, for example:

In [13]:
%%sql --module hv_genes

SELECT *
FROM ( $hv_result )
HAVING
  ( exp_mean > 6.
    AND n >= 200
    AND exp_cv > 0.5 )
ORDER BY
  exp_cv DESC

In [14]:
bq.Query(hv_genes,hv_result=r).results().to_dataframe()

,Study,HGNC_gene_symbol,n,exp_mean,exp_sigma,exp_cv
0,SARC,RPS4Y1,259,6.061873,5.548601,0.915328
1,COAD,XIST,285,6.248359,5.470363,0.875488
2,COAD,RPS4Y1,285,6.121045,5.155333,0.842231
3,BLCA,GSTM1,408,6.010392,5.006242,0.832931
4,KIRC,XIST,533,6.407005,5.271564,0.822781
5,BRCA,CPB1,1095,6.559260,5.168064,0.787904
6,LGG,RPS4Y1,516,6.704050,5.271334,0.786291
7,LGG,XIST,516,7.173960,5.539642,0.772187
8,HNSC,ACTC1,520,6.066815,4.653893,0.767106
9,BLCA,KRT6C,408,6.000740,4.567136,0.761095
